In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
# Parameters
data_dir = "rice_leaf_diseases"
image_size = (224, 224)
batch_size = 32


In [3]:
# Load datasets
train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)


Found 2596 files belonging to 5 classes.
Using 2077 files for training.
Found 2596 files belonging to 5 classes.
Using 519 files for validation.


In [4]:
class_names = train_dataset.class_names
print(class_names)

['Bacterial Blight', 'Brown Spot', 'Healthy', 'Leaf Blast', 'Tungro']


In [5]:
# Normalize pixel values
normalization_layer = layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

# Optional: Prefetch to speed up I/O
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)


In [6]:
# Data augmentation (not applied directly here, just an example)
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
])


In [ ]:
# Load base model
base_model = tf.keras.applications.VGG16(include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [ ]:
# Build model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')
])


In [ ]:
# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,847,301 (56.64 MB)

 Trainable params: 132,613 (518.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [10]:
# Train with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=20,
    callbacks=[early_stopping]
)


Epoch 1/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.4538 - loss: 1.3365 - val_accuracy: 0.7091 - val_loss: 0.7620
Epoch 2/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.7036 - loss: 0.7539 - val_accuracy: 0.7669 - val_loss: 0.6377
Epoch 3/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.7535 - loss: 0.6440 - val_accuracy: 0.7823 - val_loss: 0.5829
Epoch 4/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.7678 - loss: 0.5814 - val_accuracy: 0.7938 - val_loss: 0.5420
Epoch 5/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.7746 - loss: 0.5317 - val_accuracy: 0.7688 - val_loss: 0.5498
Epoch 6/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.7906 - loss: 0.5110 - val_accuracy: 0.8227 - val_loss: 0.4944
Epoch 7/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - accuracy: 0.8206 - loss: 0.4669 - val_accuracy: 0.8304 - val_loss: 0.4716
Epoch 8/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.8105 - loss: 0.4566 - val_accuracy: 0.8304 - v

In [ ]:
# Save model
model.save("rice_leaf_disease_model8.h5")
print("Model saved as 'rice_leaf_disease_model8.h5'.")


Model saved as 'rice_leaf_disease_model8.h5'.
